In [1]:
from selenium import webdriver
import pyautogui
import time
import random
import re
import os
import pickle

In [2]:
# 나라이름으로 된 폴더가 있는지 확인하고 없으면 만든다.
def make_directory(directory_name):
    try:
        if not(os.path.isdir(directory_name)):
            os.makedirs(os.path.join(directory_name))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print(directory_name, "폴더가 있음.")
            raise

In [3]:
# 입력 받은 크롤링 개수보다 파일이 많으면 동작하지 않음, 파일이 적을 경우 적은 만큼만 동작하여 이미지 파일의 총 개수가 
# 입력 받은 크롤링 개수가 되게 함.
def check_image_count(directory_name, total_count):
    image_list = next(os.walk('./{}/'.format(directory_name)))[2]
    image_count = len(image_list)
    
    if total_count <= image_count:
        print('입력받은 개수보다 많은 파일을 가지고 있음.')
        return False
    else:
        total_crawling_count = total_count - image_count
        max_num = 0
        max_num_index = 0
        for i in range(image_count):
            if max_num < int(re.findall('\d+', image_list[i])[0]):
                max_num = int(re.findall('\d+', image_list[i])[0])
                max_num_index = i
        # print(max_num_index, max_num)
        return total_crawling_count, max_num

In [4]:
# urls 파일 열기, shuffle
def load_urls(country_name):
    urls = []
    with open('{}_urls.txt'.format(country_name, 'r')) as file:
        for s in file:
            urls.append(s.split(' ')[0])
    random.shuffle(urls)
    return urls

In [5]:
# 각 url 마다 크롤링할 개수, 반올림하여 구함
# 데이터 전처리에서 버리는 데이터가 생기므로 더 많은 데이터 크롤링
def each_count(urls, total_crawling_count):
    each_crawling_count = total_crawling_count / len(urls)
    return round(each_crawling_count) + 2

In [6]:
# 데이터 중복 방지
# url 에서 방향을 전환시켜 나가는 방향을 바꿔준다
def edit_direction(url):
    p = re.compile('\d*\.?\d+h')
    m = p.search(url)
    direction = m.group()
    new_direction = direction.rstrip('h')
    new_direction = float(new_direction)
    new_direction += random.choice([0, 90, 180, 270])
    if new_direction >= 360:
        new_direction -= 360
    new_direction = str(new_direction) + 'h'
    url = url.replace(direction, new_direction)
    return url

In [7]:
# 각도가 80보다 낮거나 110보다 높으면 80에서 110 사이의 각도와 얼마나 차이가 나는지 리턴
# 각도가 낮으면 양수, 높으면 음수가 나온다.
# 양수가 나왔으면 밑으로 드래그, 음수가 나왔으면 위로 드래그
def edit_angle():
    url = driver.current_url
    p = re.compile('\d*\.?\d+t')
    m = p.search(url)
    check = m.group().rstrip('t')
    check = round(float(check))
    move = random.randrange(85, 111) - check
    return move    

In [8]:
# 위젯 제거.
def remove_widgets():
    remove_list = []
    remove_list.append(driver.find_element_by_class_name('widget-titlecard'))
    remove_list.append(driver.find_element_by_class_name('widget-minimap'))
    remove_list.append(driver.find_element_by_class_name('widget-compass'))
    #remove_list.append(driver.find_element_by_class_name('app-vertical-item'))
    remove_list.append(driver.find_element_by_class_name('app-horizontal-widget-holder'))
    remove_list.append(driver.find_element_by_class_name('scene-footer-container'))
    remove_list.append(driver.find_element_by_class_name('widget-watermark'))
    remove_list.append(driver.find_element_by_class_name('widget-image-header-close'))
    remove_list.append(driver.find_element_by_class_name('widget-zoom'))
    for r in remove_list:
        driver.execute_script('arguments[0].style.display="none"; return arguments[0];', r)

In [30]:
# 사진 저장
def save_screenshot(country_name, count, i):
    pyautogui.moveTo(50, 520)
    pyautogui.dragTo(1870, None, duration=1, button='left')
    for s in range(3):
        pyautogui.moveTo(x=None, y=50, duration=0.3)
        street_view = driver.find_element_by_class_name('widget-scene-canvas')
        street_view.screenshot('./{}/{}_{}.png'.format(country_name, country_name, count))
        count += 1
        i += 1
        if s == 2:
            pyautogui.moveTo(50, 520)
            pyautogui.dragTo(1870, None, duration=1, button='left')
        else:
            pyautogui.moveTo(1870, 520)
            pyautogui.dragTo(50, None, duration=1, button='left')
    return count, i

In [10]:
# 크롤러 동작

In [11]:
country_name = 'France'
make_directory(country_name)
total_crawling_count, max_num = check_image_count(country_name, 1500)
urls = load_urls(country_name)
each_crawling_count = each_count(urls, total_crawling_count)

In [12]:
print('country_name: ', country_name)
print('total_crawling_count: ', total_crawling_count)
print('max_num: ', max_num)
print('each_crawling_count: ', each_crawling_count)
print('urls: ', urls)

country_name:  France
total_crawling_count:  145
max_num:  1681
each_crawling_count:  18
urls:  ['https://www.google.com/maps/@48.8547629,2.2953509,3a,75y,307.57h,92.59t/data=!3m6!1e1!3m4!1s8RhjwS0wWnPMRHaT2Z6-Hg!2e0!7i16384!8i8192', 'https://www.google.com/maps/@48.858831,2.303515,3a,75y,240.6h,94.66t/data=!3m6!1e1!3m4!1sElb4uhUjcaT2vuovHBRYTg!2e0!7i16384!8i8192', 'https://www.google.com/maps/@48.8566074,2.3567923,3a,75y,31.27h,94.57t/data=!3m6!1e1!3m4!1sIH7WwGTUGnyMSVFQMbcdiw!2e0!7i16384!8i8192', 'https://www.google.com/maps/@48.8571097,2.3042625,3a,75y,74.93h,98.45t/data=!3m6!1e1!3m4!1so54b4Ju-Lc3ZU6vO0CRMlg!2e0!7i16384!8i8192', 'https://www.google.com/maps/@48.8606119,2.2908257,3a,75y,134.84h,100.59t/data=!3m6!1e1!3m4!1sw3cEwavyJNPqx_FhTlbn0A!2e0!7i16384!8i8192', 'https://www.google.com/maps/@48.8728291,2.3106845,3a,75y,116.44h,93.09t/data=!3m6!1e1!3m4!1s7569WmT40uOsHQFeJ2EHhQ!2e0!7i16384!8i8192', 'https://www.google.com/maps/@48.8576739,2.3804908,3a,75y,139.72h,93.48t/data=!3m6!1e

In [13]:
urls

['https://www.google.com/maps/@48.8547629,2.2953509,3a,75y,307.57h,92.59t/data=!3m6!1e1!3m4!1s8RhjwS0wWnPMRHaT2Z6-Hg!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.858831,2.303515,3a,75y,240.6h,94.66t/data=!3m6!1e1!3m4!1sElb4uhUjcaT2vuovHBRYTg!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.8566074,2.3567923,3a,75y,31.27h,94.57t/data=!3m6!1e1!3m4!1sIH7WwGTUGnyMSVFQMbcdiw!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.8571097,2.3042625,3a,75y,74.93h,98.45t/data=!3m6!1e1!3m4!1so54b4Ju-Lc3ZU6vO0CRMlg!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.8606119,2.2908257,3a,75y,134.84h,100.59t/data=!3m6!1e1!3m4!1sw3cEwavyJNPqx_FhTlbn0A!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.8728291,2.3106845,3a,75y,116.44h,93.09t/data=!3m6!1e1!3m4!1s7569WmT40uOsHQFeJ2EHhQ!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.8576739,2.3804908,3a,75y,139.72h,93.48t/data=!3m6!1e1!3m4!1sv3HXpBGVnLXbACWpsHv_sQ!2e0!7i16384!8i8192',
 'https://www.google.com/maps/@48.8564

In [14]:
random_list = [1, 1, 1, 2, 2, 3]
random_direction = ['right', 'right', 'right', 'right', 'left', 'left', 'up', 'up', 'None', 'None']

In [15]:
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver')

In [16]:
start = time.time()  # 시작 시간 저장
count = max_num + 1
# map_data 파일이 없으면 만듦
if '{}_map_data.pickle'.format(country_name) not in os.listdir('./'):
    with open('{}_map_data.pickle'.format(country_name), 'wb') as file:
        pickle.dump({}, file, pickle.HIGHEST_PROTOCOL)
    # map_data 파일이 있으면 load
with open('{}_map_data.pickle'.format(country_name), 'rb') as file:
    map_data = pickle.load(file)
    
for url in urls:
    driver.get(edit_direction(url))
    time.sleep(10)
    i = 0
    while i <= each_crawling_count:
        # 중복 체크
        data = driver.current_url.split('!')[4]
        if data in map_data:
            map_data[data] += 1
        else:
            map_data[data] = 1
        if max(map_data.values()) >= 3:
            map_data[data] -= 2
            url = random.choice(urls)
            driver.get(edit_direction(url))
            time.sleep(7)
        
        random_click = random.choice(random_list)
        if random_click == 1:
            for j in range(random.randint(4, 8)):
                pyautogui.click(x=random.randint(800, 1200), y=random.randint(700, 800))
                time.sleep(1.5)
        elif random_click == 2:
            for q in range(random.randint(3, 7)):
                pyautogui.doubleClick(x=random.randint(600, 1400), y=random.randint(600, 900))
                time.sleep(1.5)
        else:
            for k in range(2, 10):
                pyautogui.click(x=random.randint(200, 1800), y=random.randint(400, 1000))
                time.sleep(1.5)
        time.sleep(2)
        random_key = random.choice(random_direction)
        if random_key == 'up':
            pyautogui.keyDown(random_key)
            time.sleep(random.randint(6, 9))
            pyautogui.keyUp(random_key)
            time.sleep(2)
        elif random_key == 'left' or random_key == 'right':
            pyautogui.keyDown(random_key)
            time.sleep(round(random.uniform(0.7, 1.1), 1))
            pyautogui.keyUp(random_key)
        time.sleep(2)
        move = edit_angle()
        pyautogui.moveTo(960, 520)
        pyautogui.drag(None, move * 6 + 50, duration=0.3, button='left')
        remove_widgets()
        time.sleep(0.2)
        
        count, i = save_screenshot(country_name, count, i)       

# map_data save
with open('{}_map_data.pickle'.format(country_name), 'wb') as file:
    pickle.dump(map_data, file, pickle.HIGHEST_PROTOCOL)
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 2365.542107105255


In [16]:
url

'\n'

In [35]:
url.split('!')[4]

'1ssaKXEOROH81hE9FBXWl3sw'

In [18]:
with open('{}_map_data.pickle'.format(country_name), 'wb') as file:
    pickle.dump(map_data, file, pickle.HIGHEST_PROTOCOL)

In [52]:
pyautogui.moveTo(550, 520)
pyautogui.drag(None, 50, duration=0.3, button='left')
pyautogui.drag(None, 50, duration=0.2, button='left')

In [15]:
image_list = next(os.walk('./Australia/'))[2]

In [17]:
image_list.sort()

In [18]:
image_list

['Australia_1.png',
 'Australia_10.png',
 'Australia_100.png',
 'Australia_101.png',
 'Australia_102.png',
 'Australia_103.png',
 'Australia_104.png',
 'Australia_105.png',
 'Australia_106.png',
 'Australia_107.png',
 'Australia_108.png',
 'Australia_109.png',
 'Australia_11.png',
 'Australia_110.png',
 'Australia_111.png',
 'Australia_112.png',
 'Australia_113.png',
 'Australia_114.png',
 'Australia_115.png',
 'Australia_116.png',
 'Australia_117.png',
 'Australia_118.png',
 'Australia_119.png',
 'Australia_12.png',
 'Australia_120.png',
 'Australia_121.png',
 'Australia_122.png',
 'Australia_123.png',
 'Australia_124.png',
 'Australia_125.png',
 'Australia_126.png',
 'Australia_127.png',
 'Australia_128.png',
 'Australia_129.png',
 'Australia_13.png',
 'Australia_130.png',
 'Australia_131.png',
 'Australia_132.png',
 'Australia_133.png',
 'Australia_134.png',
 'Australia_135.png',
 'Australia_136.png',
 'Australia_137.png',
 'Australia_138.png',
 'Australia_139.png',
 'Australia_14.p

In [24]:
# for i in range(1, len(image_list)):
#     name = image_list[i]
#     new_name = "Korea_{}.png".format(i)
#     os.rename('Australia/{}'.format(name),'Australia/{}'.format(new_name))

In [2]:
with open('map_data.pickle', 'wb') as file:
    pickle.dump({}, file, pickle.HIGHEST_PROTOCOL)

In [ ]:
# gzip 을 이용하여 pickle 데이터를 압축 저장 할 수도 있다
import pickle
import gzip

data = {
    'a': [1, 2.0, 3, 4+6j],
    'b': ("character string", b"byte string"),
    'c': {None, True, False}
}

# save and compress.
with gzip.open('testPickleFile.pickle', 'wb') as f:
    pickle.dump(data, f)

# load and uncompress.
with gzip.open('testPickleFile.pickle','rb') as f:
    data = pickle.load(f)

In [3]:
# 중복데이터를 방지하기 위한 방법
# map_data를 저장하여 같은 장소에서 3번 크롤러가 돌 경우 -2 해주고 url를 다시 받아온다.
# map_data를 저장하여 놓았다가 크롤러를 다시 사용할때 로드하여 사용한다.
# url에서 카메라의 초기 방향을 랜덤(여기서는 보통 길이 난 방향으로 맞춰 놓았고, 좌,우,반대 방향에 길이 대부분 있으니 완전 랜덤 보다는 90, 180, 170 각도로 돌게함)으로 준다.
# url의 개수를 늘린다.

In [35]:
url = 'https://www.google.com/maps/@-33.8794048,151.2054464,3a,75y,10h,82.01t/data=!3m6!1e1!3m4!1soutl-B0oddFegWjywIC2BA!2e0!7i13312!8i6656'

In [51]:
edit_direction(url)

'https://www.google.com/maps/@-33.8794048,151.2054464,3a,75y,190.0h,82.01t/data=!3m6!1e1!3m4!1soutl-B0oddFegWjywIC2BA!2e0!7i13312!8i6656'

In [60]:
pyautogui.dragTo(500, 500)

In [61]:
for s in range(3):
    print(s)

0
1
2


In [22]:
#time.sleep(3)
pyautogui.moveTo(1900, None)
#pyautogui.dragTo(1870, None, duration=0.3, button='left')

In [39]:
time.sleep(3)
pyautogui.moveTo(50, 520)
pyautogui.dragTo(1870, None, duration=1, button='left')
time.sleep(0.5)
for s in range(3):
    pyautogui.moveTo(x=None, y=50, duration=0.3)
    time.sleep(0.5)
    if s == 2:
        pyautogui.moveTo(50, 520)
        pyautogui.dragTo(1870, None, duration=1, button='left')
    else:
        pyautogui.moveTo(1870, 520)
        pyautogui.dragTo(50, None, duration=1, button='left')

In [27]:
pyautogui.moveTo(970, 520)
pyautogui.dragTo(1870, None, duration=0.3, button='left')

In [10]:
# 수동 이미지 저장

In [23]:
# 사진 저장(수동)
def save_screenshot_2(country_name, count):
    street_view = driver.find_element_by_class_name('widget-scene-canvas')
    street_view.screenshot('./{}/{}_{}.png'.format(country_name, country_name, count))
    print('이미지 저장 완료! count: ', count)
    count += 1
    return count

In [40]:
country_name = 'United_States'
make_directory(country_name)
total_crawling_count, max_num = check_image_count(country_name, 1600)

In [41]:
print('country_name: ', country_name)
print('total_crawling_count: ', total_crawling_count)
print('max_num: ', max_num)

country_name:  United_States
total_crawling_count:  99
max_num:  1950


In [42]:
count = max_num + 1

In [43]:
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver')

In [14]:
from pynput import keyboard
from pynput.keyboard import Key

In [44]:
def on_press(key):
    #handle pressed keys
    pass

def on_release(key):
    #handle released keys
    if(key==Key.alt):
        remove_widgets()
        global count
        count = save_screenshot_2(country_name, count)

with keyboard.Listener(on_press=on_press,on_release=on_release) as listener:
    listener.join()

이미지 저장 완료! count:  1951
이미지 저장 완료! count:  1952
이미지 저장 완료! count:  1953
이미지 저장 완료! count:  1954
이미지 저장 완료! count:  1955
이미지 저장 완료! count:  1956
이미지 저장 완료! count:  1957


Unhandled exception in listener callback
Traceback (most recent call last):
  File "/home/pi/.local/lib/python3.7/site-packages/pynput/_util/__init__.py", line 162, in inner
    return f(self, *args, **kwargs)
  File "/home/pi/.local/lib/python3.7/site-packages/pynput/_util/xorg.py", line 458, in _handler
    self._handle(self._display_stop, event)
  File "/home/pi/.local/lib/python3.7/site-packages/pynput/keyboard/_xorg.py", line 552, in _handle
    self.on_release(key)
  File "/home/pi/.local/lib/python3.7/site-packages/pynput/_util/__init__.py", line 78, in inner
    if f(*args) is False:
  File "<ipython-input-44-036d72015556>", line 8, in on_release
    remove_widgets()
  File "<ipython-input-8-429a91575f24>", line 4, in remove_widgets
    remove_list.append(driver.find_element_by_class_name('widget-titlecard'))
  File "/usr/local/lib/python3.7/dist-packages/selenium/webdriver/remote/webdriver.py", line 564, in find_element_by_class_name
    return self.find_element(by=By.CLASS_NA

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=79.0.3945.79)


In [ ]:
def on_press(key):
    try:
        print('alphanumeric key {0} pressed'.format(
            key.char))
    except AttributeError:
        print('special key {0} pressed'.format(
            key))

def on_release(key):
    print('{0} released'.format(
        key))
    if key == keyboard.Key.esc:
        # Stop listener
        return False

# Collect events until released
with keyboard.Listener(
        on_press=on_press,
        on_release=on_release) as listener:
    listener.join()

# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()